In [1]:
import collections.abc
#hyper needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

In [2]:
from os import path
import os
import numpy as np
import re
#import pandas as pd
import pickle
import mne
import matplotlib.pyplot as plt

In [3]:
def from_pickle(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

In [4]:
path_raw_EEG = 'C:\\Users\\chedl\\OneDrive\\Bureau\\EMSE\\3A\\PE\\privateData\\EEG_MAHNOB_Dictinary.p'
path_raw_data = 'C:\\Users\\chedl\\OneDrive\\Bureau\\EMSE\\3A\\PE\\privateData'
all_eeg = from_pickle(path_raw_EEG)
meta = from_pickle(path.join(path_raw_data, "Metadata_dict.p"))
labels = from_pickle(path.join(path_raw_data, "Label_Dict.p"))

In [5]:
EEG_channels = ['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3',
                'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6',
                'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2']

In [6]:
labels

{2: array([4., 3.], dtype=float32),
 4: array([6., 3.], dtype=float32),
 6: array([8., 9.], dtype=float32),
 8: array([7., 5.], dtype=float32),
 10: array([6., 3.], dtype=float32),
 12: array([7., 8.], dtype=float32),
 14: array([6., 8.], dtype=float32),
 16: array([4., 7.], dtype=float32),
 18: array([9., 2.], dtype=float32),
 20: array([3., 6.], dtype=float32),
 22: array([9., 4.], dtype=float32),
 24: array([7., 3.], dtype=float32),
 26: array([7., 2.], dtype=float32),
 28: array([6., 3.], dtype=float32),
 30: array([6., 5.], dtype=float32),
 32: array([6., 2.], dtype=float32),
 34: array([2., 5.], dtype=float32),
 36: array([7., 7.], dtype=float32),
 38: array([6., 6.], dtype=float32),
 40: array([8., 8.], dtype=float32),
 132: array([3., 5.], dtype=float32),
 134: array([6., 3.], dtype=float32),
 136: array([2., 5.], dtype=float32),
 138: array([7., 2.], dtype=float32),
 140: array([4., 6.], dtype=float32),
 142: array([7., 1.], dtype=float32),
 144: array([5., 1.], dtype=float32)

In [21]:
path = 'C:\\Users\\chedl\\OneDrive\\Bureau\\EMSE\\3A\\PE\\essai'

In [7]:
# Set the path of the directory where you want to create the new directories
# Create 30 directories with names corresponding to their numbers in the specified directory\n",
for i in range(1, 31):
    dir_name = f'{i}'
    os.mkdir(os.path.join(path, dir_name))

In [8]:
# Iterate through the subject IDs and their corresponding data dictionaries
for subject_id, subject_session in meta.items():
    # Create the path to the subject directory
    subject_dir_path = os.path.join(path, str(subject_id))
    # Open the subject directory
    os.startfile(subject_dir_path)

    # Iterate through the session IDs and their corresponding data dictionaries for the current subject
    for session in subject_session['sessionId']:
        session_data = all_eeg[session]
        session_data = session_data.reshape(32, -1)
        # Create a Raw object from the preprocessed data
        raw = mne.io.RawArray(session_data, info=mne.create_info(ch_names=EEG_channels, sfreq=256))        # Create the filename for the EDF file
        edf_filename = f"recording_{session}.edf"
        # Write the Raw object to an EDF file in the current session's directory
        edf_path = os.path.join(subject_dir_path, edf_filename)
        raw.export(edf_path, overwrite=True)

ValueError: Data must be a 3D array of shape (n_epochs, n_channels, n_samples)

In [ ]:
number = 2
for directory in range(1, 31):
    dir_path = os.path.join(path, str(directory))
    print(dir_path)
    for filename in os.listdir(dir_path):
        file_path = os.path.join(dir_path, filename)
        raw = mne.io.read_raw_edf(file_path)
        match = re.search(r'\d+', filename)
        if match:
            number = int(match.group())
        my_annot = mne.Annotations(onset=[0],  # in seconds
                           duration=[120],  # in seconds, too
                           description=[str(labels[number])])
        raw.set_annotations(my_annot)
        anno = f"annotation_{number}.edf"
        # Write the Raw object to an EDF file in the current session's directory
        anno_path = os.path.join(dir_path, anno)
        raw.export(anno_path)

In [8]:
path = "C:\\Users\\chedl\\OneDrive\\Bureau\\EMSE\\3A\\PE\\essai\\2\\recording_132_raw.edf"
raw = mne.io.read_raw_edf(path)
eeg_data_ch = raw.copy().pick_channels(['O2'])
eeg_data_ch.plot( start=0, duration=1, scalings={'eeg': 60e-3})

Extracting EDF parameters from C:\Users\chedl\OneDrive\Bureau\EMSE\3A\PE\files\S001\S001R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


ValueError: No channels match the selection.

In [11]:
path = "C:\\Users\\chedl\\OneDrive\\Bureau\EMSE\\3A\\PE\\essai\\1\\annotation_2.edf"
raw = mne.io.read_raw_edf(path)
event = mne.events_from_annotations(raw)

Extracting EDF parameters from C:\Users\chedl\OneDrive\Bureau\EMSE\3A\PE\essai\1\annotation_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['[4. 3.]']


(array([[0, 0, 1]]), {'[4. 3.]': 1})

In [16]:
print(raw.times)

[0.00000000e+00 3.90625000e-03 7.81250000e-03 ... 1.19988281e+02
 1.19992188e+02 1.19996094e+02]


In [19]:
pithpath = "C:\\Users\\chedl\\OneDrive\\Bureau\EMSE\\3A\\PE\\essai\\1\\annotation_2.edf"
dir_pithpath = "C:\\Users\\chedl\\OneDrive\\Bureau\EMSE\\3A\\PE\\test"
session_data = all_eeg[2]
session_data = session_data.reshape(32, -1)
rawraw = mne.io.read_raw_edf(pithpath, preload=True)

stim_data = np.zeros((1, len(rawraw.times)))
info = mne.create_info(['STI'], rawraw.info['sfreq'], ['stim'])
stim_raw = mne.io.RawArray(stim_data, info)

rawraw.add_channels([stim_raw], force_update_info=True)# Create the filename for the EDF file
rawraw.add_events(events=[event])
print(rawraw.info['ch_names'])
edf_filename = f"recordingcoco_{2}.edf"
# Write the Raw object to an EDF file in the current session's directory
edf_path = os.path.join(dir_pithpath, edf_filename)
rawraw.export(edf_path, overwrite=True)

Extracting EDF parameters from C:\Users\chedl\OneDrive\Bureau\EMSE\3A\PE\essai\1\annotation_2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 30719  =      0.000 ...   119.996 secs...
Creating RawArray with float64 data, n_channels=1, n_times=30720
    Range : 0 ... 30719 =      0.000 ...   119.996 secs
Ready.
['Fp1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'P3', 'P7', 'PO3', 'O1', 'Oz', 'Pz', 'Fp2', 'AF4', 'Fz', 'F4', 'F8', 'FC6', 'FC2', 'Cz', 'C4', 'T8', 'CP6', 'CP2', 'P4', 'P8', 'PO4', 'O2', 'STI']
Overwriting existing file.
